In [1]:
import pandas as pd
import glob
import os
import numpy as np

In [2]:
path = r'C:\Users\Ithan Velarde\Desktop\Stage 2020\Horse Data\Small_dataset'
target_file = 'moments.json'
undesired_features = ['timestamp', 'latitude', 'longitude', 'run', 'propulsion', 'alert', 'heart_rate_variability', 'lo_cardio', 'altitude']

In [3]:
def file_extraction(path):
    directories = glob.glob(path + '\*')
    tfiles = []
    files = []
    for d in directories:
        tfiles.extend(glob.glob(d + '\*'))
    
    for f in tfiles:
        if(target_file in f):
            files.append(f)
    return files  

In [4]:
def file_to_panda(files):
    data = []
    for f in files:
        x = pd.read_json(f)
        name = os.path.splitext(f)[0]
        name += '_MIX.json'
        x.name = name
        data.append(x)
    return data

In [5]:
def select_features(d):
    y = d.drop(undesired_features, axis=1)
    y.name = d.name
    return y

In [6]:
def adapt_size_with_mean(d, maxi):
    name = d.name
    size = ((maxi - d.shape[0]), d.shape[1])
    df = pd.DataFrame(np.zeros(size), columns=d.columns)
    df=df.mask(df==0).fillna(d.mean())
    d = d.append(df, ignore_index=True)
    d.name = name
    return d

In [7]:
def longest_data_finder(data):
    maxi = 0
    for d in data:
        if(d.shape[0]>maxi):
            maxi = d.shape[0]
    return maxi

In [8]:
def add_time_index(d):
    d['time'] = range(0, d.shape[0])
    d.set_index('time')
    return d

In [9]:
def write(d):
    d.to_json(d.name)

In [10]:
files = file_extraction(path)
data = file_to_panda(files)
maxi = longest_data_finder(data)
for d in data:
    d = select_features(d)
    d = adapt_size_with_mean(d, maxi)
    write(d)